In [23]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from copy import copy
#os.chdir('../')

In [25]:
plt.rcParams['mathtext.fontset'] = 'cm'
plt.rcParams["font.family"] = "serif"
plt.rcParams["font.serif"] = ["Times New Roman"
                                                ] + plt.rcParams["font.serif"]
plt.rcParams['font.size'] = 13
plt.rcParams['figure.dpi'] = 300

# The Data

In [26]:
data = pd.read_excel('../data/data.xlsx')
data = data.drop(columns=['S/N'])
data.head()

,cDen,Pot,Sn %,pH,C2H4,CO,H2,EtoH,FORM
0,150,3.5,100,14.05,0,23,12,0,61
1,150,3.3,80,14.05,0,23,7,0,66
2,150,3.2,50,14.05,0,34,5,3,52
3,150,3.1,10,14.05,1,42,5,2,42
4,150,3.0,5,14.05,4,48,5,10,19


In [27]:
features_col = list(data.columns[:4])
target_col = list(data.columns[4:])
#target_col = [target_col[0], target_col[2]]
print('Features: ', features_col)
print('Target: ', target_col)

Features:  ['cDen', 'Pot', 'Sn %', 'pH']
Target:  ['C2H4', 'CO', 'H2', 'EtoH', 'FORM']


In [28]:
# normalize the data in target columns by 100
data[target_col] = data[target_col] / 100
data.head(2)

,cDen,Pot,Sn %,pH,C2H4,CO,H2,EtoH,FORM
0,150,3.5,100,14.05,0.0,0.23,0.12,0.0,0.61
1,150,3.3,80,14.05,0.0,0.23,0.07,0.0,0.66


In [29]:
data[features_col[2]] = data[features_col[2]] / 100
data.head(2)

,cDen,Pot,Sn %,pH,C2H4,CO,H2,EtoH,FORM
0,150,3.5,1.0,14.05,0.0,0.23,0.12,0.0,0.61
1,150,3.3,0.8,14.05,0.0,0.23,0.07,0.0,0.66


In [30]:
# create a pymatgen structure from the data. remember that it's CuSn with Sn fraction in position 2 in features_col
import pymatgen.core as pmg

def create_structure(Sn_percent):
    # create the structure
    if Sn_percent <= 1:
        base = f'Cu{1-Sn_percent}Sn{Sn_percent}'
        comp = pmg.Composition(base)
    else:
        raise ValueError('Sn percent must be less than or equal to 1')
    return comp

data['weight'] = data['Sn %'].apply(create_structure).apply(lambda x: x.weight)
data.head(5)

,cDen,Pot,Sn %,pH,C2H4,CO,H2,EtoH,FORM,weight
0,150,3.5,1.00,14.05,0.00,0.23,0.12,0.00,0.61,118.7100
1,150,3.3,0.80,14.05,0.00,0.23,0.07,0.00,0.66,107.6772
2,150,3.2,0.50,14.05,0.00,0.34,0.05,0.03,0.52,91.1280
3,150,3.1,0.10,14.05,0.01,0.42,0.05,0.02,0.42,69.0624
4,150,3.0,0.05,14.05,0.04,0.48,0.05,0.10,0.19,66.3042


In [31]:
data['Cu %'] = 1 - data['Sn %']
data.head(2)

,cDen,Pot,Sn %,pH,C2H4,CO,H2,EtoH,FORM,weight,Cu %
0,150,3.5,1.0,14.05,0.0,0.23,0.12,0.0,0.61,118.7100,0.0
1,150,3.3,0.8,14.05,0.0,0.23,0.07,0.0,0.66,107.6772,0.2


In [32]:
target_col

['C2H4', 'CO', 'H2', 'EtoH', 'FORM']

In [33]:
features_col += ['weight', 'Cu %']
X = data[features_col]
y = data[target_col]

In [34]:
from botorch.models.gp_regression import FixedNoiseGP
from botorch.models.model_list_gp_regression import ModelListGP
from botorch.models.transforms.outcome import Standardize
from gpytorch.mlls.sum_marginal_log_likelihood import SumMarginalLogLikelihood
from botorch.utils.transforms import unnormalize, normalize
from botorch.utils.sampling import draw_sobol_samples

import torch

In [35]:
X.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
cDen,35.0,269.171429,119.205824,141.000,150.00000,250.0000,350.0000,450.00
Pot,35.0,3.860000,0.500118,2.800,3.55000,4.0000,4.1500,4.70
Sn %,35.0,0.354286,0.388203,0.000,0.03000,0.1000,0.8000,1.00
pH,35.0,12.844000,2.447214,8.020,14.05000,14.0500,14.0500,14.05
weight,35.0,83.089817,21.414838,63.546,65.20092,69.0624,107.6772,118.71
Cu %,35.0,0.645714,0.388203,0.000,0.20000,0.9000,0.9700,1.00


In [36]:
import torch
from torch import Tensor
from typing import List
_VERBOSE = False
def get_experimental_bounds(X_columns: List[str], verbose: bool = _VERBOSE) -> Tensor:
    """Returns bounds of columns in X for mortar mixes.

    Args:
        X_columns: Names of the columns in the input dataset.
        verbose: Whether to print what the lower and upper bounds are set to.

    Tensor:
        A `2 x d`-dim Tensor of lower and upper mortar bounds for each column of X.
    """
    bounds_dict = {
        "cDen": (150, 450),  # in grams, as opposed to the original concrete bounds
        "Pot": (2.8, 4.7),
        "Sn %": (0, 1),
        "pH": (8.02, 14.05),
        "weight": (63.546, 118.71),
        "Cu %": (0, 1)
    }

    bounds = torch.tensor([bounds_dict[col] for col in X_columns]).T
    if verbose:
        print("The lower and upper bounds for the respective variables are set to:")
        for col, bound in zip(X_columns, bounds.T):
            print(f"\t- {col}: [{bound[0].item()}, {bound[1].item()}]")
        print()
    return bounds

In [37]:
bounds = get_experimental_bounds(features_col, verbose=False)
bounds

tensor([[150.0000,   2.8000,   0.0000,   8.0200,  63.5460,   0.0000],
        [450.0000,   4.7000,   1.0000,  14.0500, 118.7100,   1.0000]])

In [38]:
bounds.shape

torch.Size([2, 6])

In [39]:

def initialize_model(train_x, train_obj):
    # define models for objective and constraint
    train_x = normalize(train_x, bounds)
    models = []
    for i in range(train_obj.shape[-1]):
        train_y = train_obj[..., i : i + 1]
        train_yvar = torch.full_like(train_y, 0.2)
        models.append(
            FixedNoiseGP(
                train_x, train_y, train_yvar, outcome_transform=Standardize(m=1)
            )
        )
    model = ModelListGP(*models)
    mll = SumMarginalLogLikelihood(model.likelihood, model)
    return mll, model

In [40]:
X_torch, y_torch = torch.from_numpy(X.to_numpy()), torch.from_numpy(y.to_numpy())
print(X_torch.shape, y_torch.shape)

torch.Size([35, 6]) torch.Size([35, 5])


In [41]:
initialize_model(X_torch, y_torch)

/Users/gbemidebe/miniconda3/envs/MatML/lib/python3.8/site-packages/botorch/models/utils/assorted.py:173: InputDataWarning: Input data is not contained to the unit cube. Please consider min-max scaling the input data.
  warnings.warn(msg, InputDataWarning)


(SumMarginalLogLikelihood(
   (likelihood): LikelihoodList(
     (likelihoods): ModuleList(
       (0): FixedNoiseGaussianLikelihood(
         (noise_covar): FixedGaussianNoise()
       )
       (1): FixedNoiseGaussianLikelihood(
         (noise_covar): FixedGaussianNoise()
       )
       (2): FixedNoiseGaussianLikelihood(
         (noise_covar): FixedGaussianNoise()
       )
       (3): FixedNoiseGaussianLikelihood(
         (noise_covar): FixedGaussianNoise()
       )
       (4): FixedNoiseGaussianLikelihood(
         (noise_covar): FixedGaussianNoise()
       )
     )
   )
   (model): ModelListGP(
     (models): ModuleList(
       (0): FixedNoiseGP(
         (likelihood): FixedNoiseGaussianLikelihood(
           (noise_covar): FixedGaussianNoise()
         )
         (mean_module): ConstantMean()
         (covar_module): ScaleKernel(
           (base_kernel): MaternKernel(
             (lengthscale_prior): GammaPrior()
             (raw_lengthscale_constraint): Positive()
         

In [42]:
from botorch.optim.optimize import optimize_acqf, optimize_acqf_list
from botorch.acquisition.objective import GenericMCObjective
from botorch.utils.multi_objective.scalarization import get_chebyshev_scalarization
from botorch.utils.multi_objective.box_decompositions.non_dominated import (
    FastNondominatedPartitioning,
)
from botorch.acquisition.multi_objective.monte_carlo import (
    qExpectedHypervolumeImprovement,
    qNoisyExpectedHypervolumeImprovement,
)
from botorch.utils.sampling import sample_simplex

In [43]:
def dummy_function(x):
    '''return random number with shape of (n, len(target_col))'''
    
    return torch.rand(x.shape[0], len(target_col))

In [44]:
BATCH_SIZE = 4
NUM_RESTARTS = 10
RAW_SAMPLES = 512

ref_point = [0]*len(target_col)


def optimize_qehvi_and_get_observation(model, train_x, train_obj, sampler):
    """Optimizes the qEHVI acquisition function, and returns a new candidate and observation."""
    # partition non-dominated space into disjoint rectangles
    with torch.no_grad():
        pred = model.posterior(normalize(train_x, bounds)).mean
    partitioning = FastNondominatedPartitioning(
        ref_point=ref_point,
        Y=pred,
    )
    acq_func = qExpectedHypervolumeImprovement(
        model=model,
        ref_point=ref_point,
        partitioning=partitioning,
        sampler=sampler,
    )
    # optimize
    candidates, _ = optimize_acqf(
        acq_function=acq_func,
        bounds=standard_bounds,
        q=BATCH_SIZE,
        num_restarts=NUM_RESTARTS,
        raw_samples=RAW_SAMPLES,  # used for intialization heuristic
        options={"batch_limit": 5, "maxiter": 200},
        sequential=True,
    )
    # observe new values
    new_x = unnormalize(candidates.detach(), bounds=bounds)
    new_obj_true = dummy_function(new_x)
    new_obj = new_obj_true + torch.randn_like(new_obj_true, 0.2)
    return new_x, new_obj, new_obj_true

In [47]:
import time
import warnings

from botorch import fit_gpytorch_mll
from botorch.exceptions import BadInitialCandidatesWarning
from botorch.sampling.normal import SobolQMCNormalSampler
from botorch.utils.multi_objective.box_decompositions.dominated import (
    DominatedPartitioning,
)
from botorch.utils.multi_objective.pareto import is_non_dominated


warnings.filterwarnings("ignore", category=BadInitialCandidatesWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

N_BATCH = 20 
MC_SAMPLES = 16

verbose = True

hvs_qehvi, hvs_random = [], []

In [50]:
X_torch

tensor([[1.5000e+02, 3.5000e+00, 1.0000e+00, 1.4050e+01, 1.1871e+02, 0.0000e+00],
        [1.5000e+02, 3.3000e+00, 8.0000e-01, 1.4050e+01, 1.0768e+02, 2.0000e-01],
        [1.5000e+02, 3.2000e+00, 5.0000e-01, 1.4050e+01, 9.1128e+01, 5.0000e-01],
        [1.5000e+02, 3.1000e+00, 1.0000e-01, 1.4050e+01, 6.9062e+01, 9.0000e-01],
        [1.5000e+02, 3.0000e+00, 5.0000e-02, 1.4050e+01, 6.6304e+01, 9.5000e-01],
        [1.5000e+02, 3.0000e+00, 3.0000e-02, 1.4050e+01, 6.5201e+01, 9.7000e-01],
        [1.5000e+02, 2.8000e+00, 0.0000e+00, 1.4050e+01, 6.3546e+01, 1.0000e+00],
        [2.5000e+02, 4.0000e+00, 1.0000e+00, 1.4050e+01, 1.1871e+02, 0.0000e+00],
        [2.5000e+02, 3.8000e+00, 8.0000e-01, 1.4050e+01, 1.0768e+02, 2.0000e-01],
        [2.5000e+02, 3.7000e+00, 5.0000e-01, 1.4050e+01, 9.1128e+01, 5.0000e-01],
        [2.5000e+02, 3.6000e+00, 1.0000e-01, 1.4050e+01, 6.9062e+01, 9.0000e-01],
        [2.5000e+02, 3.6000e+00, 5.0000e-02, 1.4050e+01, 6.6304e+01, 9.5000e-01],
        [2.5000e

In [51]:
y_torch

tensor([[0.0000, 0.2300, 0.1200, 0.0000, 0.6100],
        [0.0000, 0.2300, 0.0700, 0.0000, 0.6600],
        [0.0000, 0.3400, 0.0500, 0.0300, 0.5200],
        [0.0100, 0.4200, 0.0500, 0.0200, 0.4200],
        [0.0400, 0.4800, 0.0500, 0.1000, 0.1900],
        [0.0700, 0.5000, 0.0500, 0.1100, 0.1400],
        [0.1500, 0.4700, 0.1100, 0.0600, 0.1100],
        [0.0000, 0.2200, 0.1200, 0.0000, 0.6300],
        [0.0000, 0.1700, 0.1000, 0.0000, 0.7000],
        [0.0200, 0.3600, 0.0800, 0.0200, 0.4800],
        [0.0600, 0.2300, 0.0700, 0.1200, 0.4200],
        [0.0800, 0.1200, 0.0700, 0.3200, 0.2300],
        [0.0900, 0.0500, 0.0600, 0.4800, 0.0900],
        [0.3100, 0.1500, 0.1500, 0.1400, 0.0900],
        [0.0000, 0.2200, 0.1200, 0.0000, 0.6300],
        [0.0000, 0.1700, 0.1000, 0.0000, 0.7000],
        [0.0200, 0.3600, 0.0800, 0.0200, 0.4800],
        [0.0600, 0.2300, 0.0700, 0.1200, 0.4200],
        [0.0800, 0.1200, 0.0700, 0.3200, 0.2300],
        [0.0900, 0.0500, 0.0600, 0.4800, 0.0900],


In [48]:
# intialize model
mll_qehvi, model_qehvi = initialize_model(X_torch, y_torch)

/Users/gbemidebe/miniconda3/envs/MatML/lib/python3.8/site-packages/botorch/models/utils/assorted.py:173: InputDataWarning: Input data is not contained to the unit cube. Please consider min-max scaling the input data.
  warnings.warn(msg, InputDataWarning)


In [49]:
mll_qehvi

SumMarginalLogLikelihood(
  (likelihood): LikelihoodList(
    (likelihoods): ModuleList(
      (0): FixedNoiseGaussianLikelihood(
        (noise_covar): FixedGaussianNoise()
      )
      (1): FixedNoiseGaussianLikelihood(
        (noise_covar): FixedGaussianNoise()
      )
      (2): FixedNoiseGaussianLikelihood(
        (noise_covar): FixedGaussianNoise()
      )
      (3): FixedNoiseGaussianLikelihood(
        (noise_covar): FixedGaussianNoise()
      )
      (4): FixedNoiseGaussianLikelihood(
        (noise_covar): FixedGaussianNoise()
      )
    )
  )
  (model): ModelListGP(
    (models): ModuleList(
      (0): FixedNoiseGP(
        (likelihood): FixedNoiseGaussianLikelihood(
          (noise_covar): FixedGaussianNoise()
        )
        (mean_module): ConstantMean()
        (covar_module): ScaleKernel(
          (base_kernel): MaternKernel(
            (lengthscale_prior): GammaPrior()
            (raw_lengthscale_constraint): Positive()
          )
          (outputscale_prior

In [ ]:
train_x_qehvi, train_obj_qehvi, train_obj_true_qehvi = (
    X_torch,
    train_obj_qparego,
    train_obj_true_qparego,
)

In [46]:




# compute hypervolume
bd = DominatedPartitioning(ref_point=ref_point, Y=y_torch)
volume = bd.compute_hypervolume().item()

hvs_qehvi.append(volume)
hvs_random.append(volume)



/Users/gbemidebe/miniconda3/envs/MatML/lib/python3.8/site-packages/botorch/models/utils/assorted.py:173: InputDataWarning: Input data is not contained to the unit cube. Please consider min-max scaling the input data.
  warnings.warn(msg, InputDataWarning)


AttributeError: 'list' object has no attribute 'shape'

In [ ]:
# run N_BATCH rounds of BayesOpt after the initial random batch
for iteration in range(1, N_BATCH + 1):

    t0 = time.monotonic()

    # fit the models
    fit_gpytorch_mll(mll_qparego)
    fit_gpytorch_mll(mll_qehvi)
    fit_gpytorch_mll(mll_qnehvi)

    # define the qEI and qNEI acquisition modules using a QMC sampler
    qparego_sampler = SobolQMCNormalSampler(sample_shape=torch.Size([MC_SAMPLES]))
    qehvi_sampler = SobolQMCNormalSampler(sample_shape=torch.Size([MC_SAMPLES]))
    qnehvi_sampler = SobolQMCNormalSampler(sample_shape=torch.Size([MC_SAMPLES]))

    # optimize acquisition functions and get new observations
    (
        new_x_qparego,
        new_obj_qparego,
        new_obj_true_qparego,
    ) = optimize_qnparego_and_get_observation(
        model_qparego, train_x_qparego, train_obj_qparego, qparego_sampler
    )
    new_x_qehvi, new_obj_qehvi, new_obj_true_qehvi = optimize_qehvi_and_get_observation(
        model_qehvi, train_x_qehvi, train_obj_qehvi, qehvi_sampler
    )
    (
        new_x_qnehvi,
        new_obj_qnehvi,
        new_obj_true_qnehvi,
    ) = optimize_qnehvi_and_get_observation(
        model_qnehvi, train_x_qnehvi, train_obj_qnehvi, qnehvi_sampler
    )
    new_x_random, new_obj_random, new_obj_true_random = generate_initial_data(
        n=BATCH_SIZE
    )

    # update training points
    train_x_qparego = torch.cat([train_x_qparego, new_x_qparego])
    train_obj_qparego = torch.cat([train_obj_qparego, new_obj_qparego])
    train_obj_true_qparego = torch.cat([train_obj_true_qparego, new_obj_true_qparego])

    train_x_qehvi = torch.cat([train_x_qehvi, new_x_qehvi])
    train_obj_qehvi = torch.cat([train_obj_qehvi, new_obj_qehvi])
    train_obj_true_qehvi = torch.cat([train_obj_true_qehvi, new_obj_true_qehvi])

    train_x_qnehvi = torch.cat([train_x_qnehvi, new_x_qnehvi])
    train_obj_qnehvi = torch.cat([train_obj_qnehvi, new_obj_qnehvi])
    train_obj_true_qnehvi = torch.cat([train_obj_true_qnehvi, new_obj_true_qnehvi])

    train_x_random = torch.cat([train_x_random, new_x_random])
    train_obj_random = torch.cat([train_obj_random, new_obj_random])
    train_obj_true_random = torch.cat([train_obj_true_random, new_obj_true_random])

    # update progress
    for hvs_list, train_obj in zip(
        (hvs_random, hvs_qparego, hvs_qehvi, hvs_qnehvi),
        (
            train_obj_true_random,
            train_obj_true_qparego,
            train_obj_true_qehvi,
            train_obj_true_qnehvi,
        ),
    ):
        # compute hypervolume
        bd = DominatedPartitioning(ref_point=problem.ref_point, Y=train_obj)
        volume = bd.compute_hypervolume().item()
        hvs_list.append(volume)

    # reinitialize the models so they are ready for fitting on next iteration
    # Note: we find improved performance from not warm starting the model hyperparameters
    # using the hyperparameters from the previous iteration
    mll_qparego, model_qparego = initialize_model(train_x_qparego, train_obj_qparego)
    mll_qehvi, model_qehvi = initialize_model(train_x_qehvi, train_obj_qehvi)
    mll_qnehvi, model_qnehvi = initialize_model(train_x_qnehvi, train_obj_qnehvi)

    t1 = time.monotonic()

    if verbose:
        print(
            f"\nBatch {iteration:>2}: Hypervolume (random, qNParEGO, qEHVI, qNEHVI) = "
            f"({hvs_random[-1]:>4.2f}, {hvs_qparego[-1]:>4.2f}, {hvs_qehvi[-1]:>4.2f}, {hvs_qnehvi[-1]:>4.2f}), "
            f"time = {t1-t0:>4.2f}.",
            end="",
        )
    else:
        print(".", end="")